In [1]:
%matplotlib notebook
import numpy as np
import cvxopt
import cvxopt.glpk

In [2]:
cvxopt.glpk.ilp?

In [3]:
## To help create the huge matrix

def unravel(i,j,k,order=3):
    n2 = order*order
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)

In [28]:
def sudokuConstraints(order=3):
    n2 = order*order
    A = np.zeros((4*n2*n2,n2*n2*n2))
    ## line constraints: only one number per line
    c=0
    for k in range(n2): ## for all numbers
        for j in range(n2): ## for all columns
            for i in range(n2): 
                A[c,unravel(i,j,k)] = 1 ## only one number k on line i
            c += 1
        
    ## column constraints: only one number per column
    for k in range(n2):
        for i in range(n2):
            for j in range(n2):
                A[c,unravel(i,j,k)] = 1
            c += 1
        
    # unicity constraints
    for i in range(n2): ## for all rows
        for j in range(n2): ## for all columns
            for k in range(n2):
                A[c, unravel(i,j,k)] = 1 ## only one number in each position
            c += 1
        
    # square constraints
    for k in range(n2): ## for all numbers
        for i in range(order): ## for all row square
            for j in range(order): ## for all columns squares
                for u in range(order): ## all the lines in the subsquare
                    for v in range(order): ## all the columns in the subsquare
                        A[c, unravel(i*order+u,j*order+v,k)]=1
                c += 1
            
    print("Total number of constraints=", c)
    return(A)

    
def sanityCheck(A,order=3):
    n2 = order*order
    c = 4*(order*order)**2
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

A = sudokuConstraints(order=3)
sanityCheck(A)

Total number of constraints= 324
All constraints OK


In [29]:
## add the fixed numbers constraints
K = np.array([
[0,8,0, 9,0,1, 0,5,0],
[0,0,2, 6,8,7, 3,0,0],
[0,0,3, 0,0,0, 6,0,0],
[3,9,0, 0,0,0, 0,6,5],
[6,0,0, 4,7,5, 0,0,3],
[5,7,0, 0,0,0, 0,8,4],
[0,0,9, 0,0,0, 8,0,0],
[0,0,5, 1,2,4, 9,0,0],
[0,4,0, 8,0,3, 0,2,0]])


A=sudokuConstraints(order=3)
for i in range(n2):
    for j in range(n2):
        k = K[i,j]
        if (k>0):
            newrow=np.zeros(n2*n2*n2)
            newrow[unravel(i,j,k-1)]=1
            A=np.vstack((A,newrow))
            
print("A.shape=", A.shape)

Total number of constraints= 324
A.shape= (359, 729)


In [33]:
## solving
from cvxopt import matrix

def solveSudoku(A):
    b=matrix(np.ones(A.shape[0])) ## set partition
    c=matrix(np.zeros(A.shape[1])) ## zero cost
    G=matrix(np.zeros(A.shape))
    h=matrix(np.zeros(A.shape[0]))
    binary=np.array(range(A.shape[1]))
    I=set(binary)
    B=set(binary)
    Aeq = matrix(A)
    (status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))
    return(status,solution)

In [32]:
## print solution
def printsol(sol):
    sep3="+-----+-----+-----+"
    for i in range(n2):
        if (i%3 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    print(k+1,end='')
            if (j%3 ==2):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)
        
printsol(solution)
          

+-----+-----+-----+
|7 8 6|9 3 1|4 5 2|
|4 5 2|6 8 7|3 1 9|
|9 1 3|5 4 2|6 7 8|
+-----+-----+-----+
|3 9 4|2 1 8|7 6 5|
|6 2 8|4 7 5|1 9 3|
|5 7 1|3 6 9|2 8 4|
+-----+-----+-----+
|2 3 9|7 5 6|8 4 1|
|8 6 5|1 2 4|9 3 7|
|1 4 7|8 9 3|5 2 6|
+-----+-----+-----+


## Harder Sudoku

In [36]:
## add the fixed numbers constraints
K = np.array([
[7,0,0, 0,0,0, 4,0,0],
[0,2,0, 0,7,0, 0,8,0],
[0,0,3, 0,0,8, 0,0,9],
[0,0,0, 5,0,0, 3,0,0],
[0,6,0, 0,2,0, 0,9,0],
[0,0,1, 0,0,7, 0,0,6],
[0,0,0, 3,0,0, 9,0,0],
[0,3,0, 0,4,0, 0,6,0],
[0,0,9, 0,0,1, 0,0,5]])

A=sudokuConstraints()

## add fixed constraintes
for i in range(n2):
    for j in range(n2):
        k = K[i,j]
        if (k>0):
            newrow=np.zeros(n2*n2*n2)
            newrow[unravel(i,j,k-1)]=1
            A=np.vstack((A,newrow))
            
status,solution=solveSudoku(A)
printsol(solution)

Total number of constraints= 324
+-----+-----+-----+
|7 9 8|6 3 5|4 2 1|
|1 2 6|9 7 4|5 8 3|
|4 5 3|2 1 8|6 7 9|
+-----+-----+-----+
|9 7 2|5 8 6|3 1 4|
|5 6 4|1 2 3|8 9 7|
|3 8 1|4 9 7|2 5 6|
+-----+-----+-----+
|6 1 7|3 5 2|9 4 8|
|8 3 5|7 4 9|1 6 2|
|2 4 9|8 6 1|7 3 5|
+-----+-----+-----+
